In [1]:
import tensorflow as tf
from keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split
from glob import glob
import os
from tqdm import tqdm
from datetime import datetime
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [2]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [7]:
w, h = 15, 15
base_path = '/content/drive/MyDrive/AI-Project/dataset-part1/*npz'

file_list = glob(base_path)
x_data, y_data = [], []
for file_path in tqdm(file_list):
    data = np.load(file_path)
    x_data.extend(data['inputs'])
    y_data.extend(data['outputs'])

x_data = np.array(x_data, np.float32).reshape((-1, h, w, 1))
y_data = np.array(y_data, np.float32).reshape((-1, h * w))

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=2020)

del x_data, y_data

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

100%|██████████| 2001/2001 [00:08<00:00, 248.29it/s]


(938476, 15, 15, 1) (938476, 225)
(234620, 15, 15, 1) (234620, 225)


In [8]:
model = models.Sequential([
    layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=(h, w, 1)),
    layers.Conv2D(128, 7, activation='relu', padding='same'),
    layers.Conv2D(256, 7, activation='relu', padding='same'),
    layers.Conv2D(128, 7, activation='relu', padding='same'),
    layers.Conv2D(64, 7, activation='relu', padding='same'),
    layers.Conv2D(1, 1, activation=None, padding='same'),
    layers.Reshape((h * w,))
])

model.add(layers.Activation('sigmoid'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 15, 15, 64)        3200      
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 128)       401536    
                                                                 
 conv2d_2 (Conv2D)           (None, 15, 15, 256)       1605888   
                                                                 
 conv2d_3 (Conv2D)           (None, 15, 15, 128)       1605760   
                                                                 
 conv2d_4 (Conv2D)           (None, 15, 15, 64)        401472    
                                                                 
 conv2d_5 (Conv2D)           (None, 15, 15, 1)         65        
                                                                 
 reshape (Reshape)           (None, 225)               0

- kernel 7,5,3,3,3: 53.3
- kernel all 7: 56.12
- kernel all 7, double kernel:

In [9]:
start_time = datetime.now().strftime('%Y%m%d_%H%M%S')
os.makedirs('models', exist_ok=True)

model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=20,
    callbacks=[
        ModelCheckpoint('./models/%s.h5' % (start_time), monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, mode='auto')
    ],
    validation_data=(x_val, y_val),
    use_multiprocessing=True,
    workers=16
)

Epoch 1/20
3666/3666 [==============================] - ETA: 0s - loss: 0.0176 - acc: 0.3193
Epoch 1: val_acc improved from -inf to 0.45142, saving model to ./models/20231219_004933.h5
3666/3666 [==============================] - 148s 38ms/step - loss: 0.0176 - acc: 0.3193 - val_loss: 0.0130 - val_acc: 0.4514 - lr: 0.0010
Epoch 2/20
   1/3666 [..............................] - ETA: 2:00 - loss: 0.0123 - acc: 0.4961

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3666/3666 [==============================] - ETA: 0s - loss: 0.0122 - acc: 0.4796
Epoch 2: val_acc improved from 0.45142 to 0.50222, saving model to ./models/20231219_004933.h5
3666/3666 [==============================] - 130s 35ms/step - loss: 0.0122 - acc: 0.4796 - val_loss: 0.0114 - val_acc: 0.5022 - lr: 0.0010
Epoch 3/20
3666/3666 [==============================] - ETA: 0s - loss: 0.0108 - acc: 0.5241
Epoch 3: val_acc improved from 0.50222 to 0.52761, saving model to ./models/20231219_004933.h5
3666/3666 [==============================] - 129s 35ms/step - loss: 0.0108 - acc: 0.5241 - val_loss: 0.0107 - val_acc: 0.5276 - lr: 0.0010
Epoch 4/20
3665/3666 [============================>.] - ETA: 0s - loss: 0.0099 - acc: 0.5597
Epoch 4: val_acc improved from 0.52761 to 0.54860, saving model to ./models/20231219_004933.h5
3666/3666 [==============================] - 129s 35ms/step - loss: 0.0099 - acc: 0.5597 - val_loss: 0.0102 - val_acc: 0.5486 - lr: 0.0010
Epoch 5/20
3666/3666 [========

In [10]:
model.save('/content/drive/MyDrive/AI_Project_model/model', save_format="h5")

In [11]:
w, h = 15, 15
base_path = '/content/drive/MyDrive/AI-Project/dataset-part2/*npz'

file_list = glob(base_path)
x_data, y_data = [], []
for file_path in tqdm(file_list):
    data = np.load(file_path)
    x_data.extend(data['inputs'])
    y_data.extend(data['outputs'])

x_data = np.array(x_data, np.float32).reshape((-1, h, w, 1))
y_data = np.array(y_data, np.float32).reshape((-1, h * w))

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=2020)

del x_data, y_data

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

100%|██████████| 2000/2000 [00:07<00:00, 262.54it/s]


(1012195, 15, 15, 1) (1012195, 225)
(253049, 15, 15, 1) (253049, 225)


In [16]:
from keras.models import load_model

model = load_model('/content/drive/MyDrive/AI_Project_model/model')
start_time = datetime.now().strftime('%Y%m%d_%H%M%S')
model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=20,
    callbacks=[
        ModelCheckpoint('./models/%s.h5' % (start_time), monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, mode='auto')
    ],
    validation_data=(x_val, y_val),
    use_multiprocessing=True,
    workers=16
)

Epoch 1/20
3954/3954 [==============================] - ETA: 0s - loss: 0.0119 - acc: 0.4868
Epoch 1: val_acc improved from -inf to 0.51711, saving model to ./models/20231219_013916.h5
3954/3954 [==============================] - 156s 39ms/step - loss: 0.0119 - acc: 0.4868 - val_loss: 0.0108 - val_acc: 0.5171 - lr: 0.0010
Epoch 2/20
   1/3954 [..............................] - ETA: 2:23 - loss: 0.0108 - acc: 0.5352

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3954/3954 [==============================] - ETA: 0s - loss: 0.0102 - acc: 0.5465
Epoch 2: val_acc improved from 0.51711 to 0.54478, saving model to ./models/20231219_013916.h5
3954/3954 [==============================] - 139s 35ms/step - loss: 0.0102 - acc: 0.5465 - val_loss: 0.0104 - val_acc: 0.5448 - lr: 0.0010
Epoch 3/20
3953/3954 [============================>.] - ETA: 0s - loss: 0.0093 - acc: 0.5915
Epoch 3: val_acc improved from 0.54478 to 0.56540, saving model to ./models/20231219_013916.h5
3954/3954 [==============================] - 139s 35ms/step - loss: 0.0093 - acc: 0.5915 - val_loss: 0.0099 - val_acc: 0.5654 - lr: 0.0010
Epoch 4/20
3953/3954 [============================>.] - ETA: 0s - loss: 0.0085 - acc: 0.6323
Epoch 4: val_acc improved from 0.56540 to 0.57993, saving model to ./models/20231219_013916.h5
3954/3954 [==============================] - 139s 35ms/step - loss: 0.0085 - acc: 0.6323 - val_loss: 0.0097 - val_acc: 0.5799 - lr: 0.0010
Epoch 5/20
3953/3954 [========

In [17]:
model.save('/content/drive/MyDrive/AI_Project_model/model2', save_format="h5")

In [3]:
w, h = 15, 15
base_path = '/content/drive/MyDrive/AI-Project/dataset-part3/*npz'

file_list = glob(base_path)
x_data, y_data = [], []
for file_path in tqdm(file_list):
    data = np.load(file_path)
    x_data.extend(data['inputs'])
    y_data.extend(data['outputs'])

x_data = np.array(x_data, np.float32).reshape((-1, h, w, 1))
y_data = np.array(y_data, np.float32).reshape((-1, h * w))

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=2020)

del x_data, y_data

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

100%|██████████| 2000/2000 [00:15<00:00, 127.41it/s]


(1284432, 15, 15, 1) (1284432, 225)
(321108, 15, 15, 1) (321108, 225)


In [5]:
from keras.models import load_model

model = load_model('/content/drive/MyDrive/AI_Project_model/model2')
start_time = datetime.now().strftime('%Y%m%d_%H%M%S')
model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=30,
    callbacks=[
        ModelCheckpoint('./models/%s.h5' % (start_time), monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.02, patience=5, verbose=1, mode='auto')
    ],
    validation_data=(x_val, y_val),
    use_multiprocessing=True,
    workers=16
)

Epoch 1/30
5018/5018 [==============================] - ETA: 0s - loss: 0.0125 - acc: 0.4635
Epoch 1: val_acc improved from -inf to 0.49996, saving model to ./models/20231219_022845.h5
5018/5018 [==============================] - 195s 37ms/step - loss: 0.0125 - acc: 0.4635 - val_loss: 0.0112 - val_acc: 0.5000 - lr: 0.0010
Epoch 2/30
   1/5018 [..............................] - ETA: 3:04 - loss: 0.0104 - acc: 0.5430

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5017/5018 [============================>.] - ETA: 0s - loss: 0.0106 - acc: 0.5252
Epoch 2: val_acc improved from 0.49996 to 0.52846, saving model to ./models/20231219_022845.h5
5018/5018 [==============================] - 184s 37ms/step - loss: 0.0106 - acc: 0.5252 - val_loss: 0.0106 - val_acc: 0.5285 - lr: 0.0010
Epoch 3/30
5017/5018 [============================>.] - ETA: 0s - loss: 0.0098 - acc: 0.5658
Epoch 3: val_acc improved from 0.52846 to 0.54768, saving model to ./models/20231219_022845.h5
5018/5018 [==============================] - 177s 35ms/step - loss: 0.0098 - acc: 0.5658 - val_loss: 0.0102 - val_acc: 0.5477 - lr: 0.0010
Epoch 4/30
5017/5018 [============================>.] - ETA: 0s - loss: 0.0091 - acc: 0.6017
Epoch 4: val_acc improved from 0.54768 to 0.56247, saving model to ./models/20231219_022845.h5
5018/5018 [==============================] - 183s 36ms/step - loss: 0.0091 - acc: 0.6017 - val_loss: 0.0100 - val_acc: 0.5625 - lr: 0.0010
Epoch 5/30
5017/5018 [========

In [6]:
model.save('/content/drive/MyDrive/AI_Project_model/model3', save_format="h5")

In [ ]:
w, h = 15, 15
base_path = '/content/drive/MyDrive/AI-Project/dataset-part4/*npz'

file_list = glob(base_path)
x_data, y_data = [], []
for file_path in tqdm(file_list):
    data = np.load(file_path)
    x_data.extend(data['inputs'])
    y_data.extend(data['outputs'])

x_data = np.array(x_data, np.float32).reshape((-1, h, w, 1))
y_data = np.array(y_data, np.float32).reshape((-1, h * w))

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=2020)

del x_data, y_data

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

In [ ]:
model = load_model('/content/drive/MyDrive/AI_Project_model/model3')
start_time = datetime.now().strftime('%Y%m%d_%H%M%S')
model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=30,
    callbacks=[
        ModelCheckpoint('./models/%s.h5' % (start_time), monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, mode='auto')
    ],
    validation_data=(x_val, y_val),
    use_multiprocessing=True,
    workers=16
)

In [ ]:
model.save('/content/drive/MyDrive/AI_Project_model/model4', save_format="h5")

In [ ]:
i = 0

for y in range(h):
    for x in range(w):
        print('%2d' % x_val[i][y, x], end='')
    print()

 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0-1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [ ]:
y_pred = model.predict(np.expand_dims(x_val[i], axis=0)).squeeze()
y_pred = y_pred.reshape((h, w))

y, x = np.unravel_index(np.argmax(y_pred), y_pred.shape)

print(x, y, y_pred[y, x])

1/1 [==============================] - 0s 346ms/step
6 12 0.9939042
